### Utility Methods
This method is used to bridge the difference in the port collection frequency between application QoE evaluation platform and WiFi metrics collection platform. Since application metrics are collected at twice more frequently i.e. every quarter of a second, this method rounds up the time stamp half a second duration and then we can get average values of MOS score, TI & SI values by grouping the QoE metrics data by the rounded timestamp. 

In [22]:
def custom_round(value):
    # Round up to one decimal point
    rounded_value = np.ceil(value * 10) / 10
    # Check if first decimal is less than 5, set it to 0
    if (rounded_value * 10) % 10 <= 5:
        return np.floor(rounded_value)
    else:
        return np.floor(rounded_value) + 0.5

Following two methods parse "port_metrics.csv" and "vap_metrics.csv" file respectively and returns a data frame with interested KPIs.

In [20]:
def parse_port_metrics(file):
    datagram = pd.read_csv(file)
    cols = ['activity', 'noise', 'rx-rate', 'tx-rate', 'beacon', 'bps rx',
            'bps rx ll', 'bps tx', 'bps tx ll', 'channel', 'collisions', 'connections', 'pps rx', 'pps tx', 'qlen', 'retry failed', 'rx crc',
        'rx drop', 'rx errors', 'rx fifo', 'rx frame', 'rx length', 'rx miss', 'rx pkts', 'signal', 'tx pkts',
        'rx over', 'tx abort', 'tx crr', 'tx errors', 'tx fifo', 'tx hb',
        'tx wind', 'tx-failed %', 'wifi retries', 'timestamp']
    df = datagram.loc[:, cols]
    df['db'] = df['noise'].str.extract('(^[-+]?\d+)').astype(float)
    df['sig'] = df['signal'].str.extract('(^[-+]?\d+)').astype(float)
    df['rxrate'] = df['rx-rate'].str.extract('(^[-+]?\d+)').astype(float)
    df['txrate'] = df['tx-rate'].str.extract('(^[-+]?\d+)').astype(float)
    df = df.drop(['noise', 'rx-rate', 'tx-rate', 'signal'], axis = 1)
    return df

In [21]:
def parse_vap_metrics(file):
    data = pd.read_csv(file)
    col_map = {'idle': 'sta_idle', 'signal': 'sta_signal', 'signal avg': 'sta_signal_avg',
               'tx retries': 'sta_tx_retries', 'tx-failed': 'sta_tx_failed'}
    cols = ['idle', 'rx rate', 'signal', 'signal avg', 'tx rate', 'tx retries', 'tx-failed', 'tx pkts', 'rx pkts', 'station bssid', 'timestamp']
    df = data.loc[:, cols]
    df.rename(columns = col_map, inplace = True)
    df['sta_rx_rate'] = df['rx rate'].str.extract('(^[-+]?\d+)').astype(float)
    df['sta_tx_rate'] = df['tx rate'].str.extract('(^[-+]?\d+)').astype(float)
    df['sta_tx_pkts'] = df['tx pkts']
    df['sta_rx_pkts'] = df['rx pkts']
    df['sta_bssid'] = df['station bssid']
    df = df.drop(['rx rate', 'tx rate', 'tx pkts', 'rx pkts', 'station bssid'], axis = 1)
    return df

This method merges QoE metrics with WiFi metrics. Each Application QOE metrics data set (row) is duplicated as many virtual STAs used in a particular experiment instance and then merged with already merged Port and VAP metrics data frame.

In [24]:
def merge_qoe_df(port_df, qoe_df):
    ts_count = port_df.groupby('timestamp')['stas'].count().shape[0]
    if all(col in qoe_df.columns for col in ['MOS', 'SI', 'TI']):
        grouped_qoe = qoe_df.groupby('ts')[['MOS', 'SI', 'TI']].mean().round(1).reset_index()
        #grouped_qoe.rename(columns = {'ts': 'timestamp'}, inplace = True)
        #print(grouped_qoe.shape, ts_count)
        if grouped_qoe.shape[0] > ts_count:
            merge_qoe = grouped_qoe.iloc[: int(ts_count - grouped_qoe.shape[0])]
        elif grouped_qoe.shape[0] < ts_count:
            return None
        else:
            merge_qoe = grouped_qoe.copy()

        # Empty list to store the repeated rows
        repeated_rows = []
        # Iterate through the Series and df_left
        for idx, value in enumerate(port_df.groupby('timestamp')['stas'].count()):
            # Repeat the row value times
            repeated_row = pd.DataFrame(np.repeat(merge_qoe.iloc[[idx]], value, axis = 0), columns = merge_qoe.columns)
            repeated_rows.append(repeated_row)

        # Concatenate all the repeated rows
        qoe_repeated = pd.concat(repeated_rows, ignore_index=True)
        #print(f"# QoE concated shape {qoe_repeated.shape} #")
        # Merge the repeated left DataFrame with the right DataFrame
        df_merged = pd.merge(port_df, qoe_repeated, left_index=True, right_index=True)
        #print(f"# Merged QoE DF shape {df_merged.shape} #")

    # vSTAs with RX rate less than or equal to 6.0 are dummy vSTAs, so drop those rows.
    return df_merged.query('sta_rx_rate > 6.0')

### Recursive WiFi metrics, QoE metrics merging utility function
This method is the root method that walks through the data directory and merges Wifi port metrics with virtual STA metrics as well as application QoE metrics. Merged data frames are returned as a map between the root path (experiment instance) and the data frame. 

In [23]:
def parse_port_vap_metrics(base):
    dataframes = dict()
    for root, dirs, files in os.walk(base):
        pf = 'port_metrics.csv' in files
        vf = 'vap_metrics.csv' in files
        qf = 'Qoe.xlsx' in files

        # If CSV files exist in the current directory
        row_repeat = 11
        if pf and vf and qf:
            pm_df = parse_port_metrics(os.path.join(root, 'port_metrics.csv')).sort_values(by = 'timestamp', ascending = True)
            vm_df = parse_vap_metrics(os.path.join(root, 'vap_metrics.csv')).sort_values(by = 'timestamp', ascending = True)

            splits = root.split('/')
            vm_df['direction'] = splits[3]
            vm_df['distance'] = splits[5]
            vm_df['sta_bw'] = int(re.search(r'^\d+', splits[6]).group()) if re.search(r'^\d+', splits[6]) is not None else 0
            vm_df['stas'] = int(re.search(r'^\d+', splits[7]).group()) if re.search(r'^\d+', splits[7]) is not None else 0

            repeated_rows = []
            for idx, value in enumerate(vm_df.groupby('timestamp')['sta_bssid'].count()):
                repeated_row = pd.DataFrame(np.repeat(pm_df.iloc[[idx]], value, axis = 0), columns = pm_df.columns)
                repeated_rows.append(repeated_row)

            pm_r = pd.concat(repeated_rows, ignore_index=True).drop('timestamp', axis = 1)
            #print(f"## PM DF concated shape: {pm_r.shape} ##")
            
            merged_df = pd.merge(pm_r, vm_df, left_index=True, right_index=True)
            #print(f"## Merged DF shape {merged_df.shape} ##")
            sorted_df = merged_df.sort_values(by='timestamp', ascending = True)

            qoe_xlsx = pd.read_excel(os.path.join(root, 'Qoe.xlsx'), sheet_name = 'Raw Data 1')
            qoe_n = qoe_xlsx.loc[:, ['TimeStamp', 'MOS', 'SI', 'TI']]
            qoe_n['ts'] = qoe_n['TimeStamp'].apply(custom_round)

            qoe_n['MOS_R'] = qoe_n['MOS'].round(1)
            qoe_n.drop(['TimeStamp', 'MOS'], axis = 1, inplace = True)
            qoe_n.rename(columns = {'MOS_R': 'MOS'}, inplace = True)

            if qoe_n.groupby('ts')['MOS'].count()[0] >= 120:
                qoe = qoe_n.query('ts >= 1 and ts <= 62')
            else:
                qoe = qoe_n.copy()

            df =  merge_qoe_df(sorted_df, qoe)
            if df is not None:
                dataframes[root] = df

    return dataframes